In [2]:
# coding: utf-8
# encoding：utf-8
import numpy as np
import pandas as pd
import uuid
import time 

 
    
def getUid():
    code = str(uuid.uuid1())#基于时间戳生成通用唯一标识符
    abc = code.split('-')
    delimiter = ''#分隔符
    code = delimiter.join(abc)  #id
    return code
#三元组
sub = []
pre = []
obj = []

def tupleInsertBatch(uid, value, attr_dict, attr):#用于基本属性
    for i in range(len(value)):
        #append 属性，添加行/列；Append rows of other to the end of this frame, returning a new object
        sub.append(uid)
        pre.append(attr_dict[attr[i]])#从基本属性字典中选取属性
        obj.append(value[i])#表中的数据
            
def tupleInsert(subject, predicate, objectx):
    sub.append(subject)
    pre.append(predicate)
    obj.append(objectx)

#load T81data
td81_data= pd.read_csv('td81.csv', header = 0)
#print 'T81data shape is (%d,%d)'%(td81_data.shape[0],td81_data.shape[1])
#print 'The T81data columns type is :\n %s ' %(td81_data.dtypes)
        
cust=pd.read_csv('customer.csv', header = 0)
cust.columns = ["Cust_Id","name","idt","risk_id","rate"]
        
#前缀
prefix= '<http://www.cmbchina.com/bigdata/financial_risk#'
pre_type = '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
        

############校验
#创建异常类并初始化 

class DatapreproError(Exception):
    def __init__ (self,err):
                Exception.__init__(self)#调用基类的__init__进行初始化
                self.err=err
                
#预处理表格分割合并check
'''try :
    ary=np.array(td_1['Cust_Id']-td_2['Cust_Id']).T
    for i in range (ary.shape[0]):
                if ary[i]!=1:
                    raise DatapreproError('数据预处理异常')           
except DatapreproError,e:
    print e.err
else:
    print 'data preprocessing'
'''
def Combine_check(ary):
    for i in range (ary.shape[0]):
                if ary[i]!=0:
                    #print '数据预处理异常,请检查第%d行数据' %i
                    raise DatapreproError('The tot_lia and tot_ass from TD are mismatch at %d row' %i)
                else:
                    pass
    print 'table is ready'

#max取值校验
def Max_prio_check(table):
    rel=table.duplicated('Cust_Id',keep='first')
    for i in range (table.shape[0]):
        if rel[i]==1:#true/1表示有重复
            #print 'idxmax func error'
            raise DatapreproError("idxmax func error")
        else:
            pass 
    print 'max priority selecting check done'
        
#df.duplicated('col1',False)
#输出：Series([True,True,True,True,True,False,False],index=['a','a','b','c','b','a','c']) 



        
####################（0）
#########pre-process
#数据处理
#表一
td_1=td81_data[td81_data.Row_Id_Nm.isin(['负债合计'])]
td_1=td_1.reset_index(drop=True)
#td_1['priority_new']=(td_1['Cust_Id'].astype(str)+td_1['Rpt_Id'].astype(str)).astype(int)#Rpt_Id是否是数字/字母，有待确认
#或者直接不用优先级，而直接用 df.sort_values(by=['col1', 'col2']) 多重排序
td_1_result= td_1.sort_values(by=['Cust_Id','Rpt_Id'])#排序为了表一表二的行一一对应
td_1['tot_lia']=td_1_result['End_Tm_Val']#负债合计

#表二
td_2=td81_data[td81_data.Row_Id_Nm.isin(['负债和所有者权益']or['负债和所有者权益()'])]
td_2=td_2.reset_index(drop=True)
td_2_result= td_2.sort_values(by=['Cust_Id','Rpt_Id'])
td_2['tot_ass']=td_2_result['End_Tm_Val']#负债和所有者权益

##分割后对比，确保不错行
#print np.array([td_1['Cust_Id'],td_2['Cust_Id'],(td_1['Cust_Id']-td_2['Cust_Id'])]).T
       
#新表
td= pd.concat([td_1, td_2['tot_ass']],axis=1)
print td
#print 'The td shape is (%d,%d)' %(td.shape[0],td.shape[1])

chk=np.array(td_1['Cust_Id']-td_2['Cust_Id']).T
Combine_check(chk)

    
#提取排序后max数据用于计算负债率
#http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
'''
td['priority'][td.Rpt_Clb_Cd=="合并"] = '2'
td['priority'][td.Rpt_Clb_Cd=="汇总"] = '1'
td['priority'][td.Rpt_Clb_Cd=="本部"] = '0'
td['priority'][td.Aud_Ind=="是"] =  td['priority'] + '1'
td['priority'][td.Aud_Ind=="否"] =  td['priority'] + '0'
td['priority'][td.Rpt_Afl_Mdl=="new"] =  td['priority'] + '1'
td['priority'][td.Rpt_Afl_Mdl=="old"] =  td['priority'] + '0'
'''
#https://jingyan.baidu.com/article/0eb457e508b6d303f0a90572.html
#iloc——通过行号获取行数据 
#loc——通过列名获取行数据 
#df.loc[df['A'] > 2, 'B'] = new_val  ; is  the same of df['B'][df.A > 2]=new_val
td['priority']='a'
td.loc[td['Rpt_Clb_Cd']=="合并",'priority']='2'
td.loc[td['Rpt_Clb_Cd']=="汇总",'priority']='1'
td.loc[td['Rpt_Clb_Cd']=="本部",'priority']='0'
td.loc[td['Aud_Ind']=="是",'priority']=td['priority'] + '1'
td.loc[td['Aud_Ind']=="否",'priority']=td['priority'] + '0'
td.loc[td['Rpt_Afl_Mdl']=="new",'priority']=td['priority'] + '1'
td.loc[td['Rpt_Afl_Mdl']=="old",'priority']=td['priority'] + '0'
td['priority']=td['priority'].astype(int)
#print td.dtypes
#td.to_csv('before.csv', index = False, sep = ',', header = False, encoding='utf_8')
td = td.loc[td.groupby('Cust_Id')['priority'].idxmax()].reset_index()
#td.to_csv('after.csv', index = False, sep = ',', header = False, encoding='utf_8')
del td['priority']

Max_prio_check(td)


tsf_lia=td['tot_lia'].str.replace(",","").astype(float)
tsf_ass=td['tot_ass'].str.replace(",", "").astype(float)

td['la_rate']=tsf_lia/tsf_ass#25
print 'liabilities ratio calculating done'

td['la_rate']=td['la_rate'].astype(float)
td['tot_lia']=tsf_lia #使得tot_lia转化为float
td['tot_ass']=tsf_ass
#print td.shape
td['uidRpt']='a'#26
td['uidLia']='a'
td['uidAss']='a'
td['uidLarate']='a'#29，but is the 30 column,  from 0
#print td.shape
#添加uid
for i in range(td.shape[0]):
    for j in range(26, 30):#26-29
              code = getUid()
              td.iloc[i,j] = code   
#最终用于生成三元组的表Fina
Fina=td
#print 'Data shape for transferring to triple is (%d,%d)' %(Fina.shape[0],Fina.shape[1])

#######################（1）
#Industry 对象构建
idtcd=prefix + 'Idt_Cd>'
idtnm=prefix + 'Idt_Nm>'
idt_clea=cust['idt']
        
#提取ind的种类及uid
idt_clea=idt_clea.drop_duplicates()
idt_clea=idt_clea.reset_index(drop=True)
N=idt_clea.shape[0]
        
temp=pd.DataFrame(np.random.rand(N,1))
temp['idt_cd'],temp['idt_nm']=idt_clea.str.split(':',1).str#更具冒号进行分割
del temp[0]
#print temp['idt_cd']

temp["uidIdt"] ='a'    
for i in range(N):#N对应industry类中的N个行业对象
    code = getUid()
    uid = prefix + str(code) + '>'
    temp.iloc[i,2] = code#第3列即为uidInd 
            
    tupleInsert(uid, pre_type, prefix + 'IronSteel>')
    tupleInsert(uid,idtcd ,  '"'+temp.iloc[i,0]+'"') 
    #注意csv格式，双引号用两个单引号表示  https://www.cnblogs.com/xh0102/p/5244190.html
    tupleInsert(uid, idtnm, '"'+temp.iloc[i,1]+'"') #第3列是uidInd
print 'industry creating done'
#print pd.DataFrame([sub, pre, obj]).head(3)
#########################（2）
#创建customer 对象
cust["uidCust"] = 'a' 
for i in range(cust.shape[0]):
    code = getUid()
    cust.iloc[i,5] = code#第6列即为uidCust
#属性property
Cust_Id = prefix + 'Cust_Core_Id>'
risk_id = prefix + 'Cust_Rsk_Id>'
idt = prefix + 'Cust_Idt_Typ>'
name = prefix + 'Cust_Nm>'
rate = prefix + 'Cust_Rat>'
#关系relation
hasIdtTyp = prefix + 'hasIdtTyp>'
attr_dict = {'Cust_Id' : Cust_Id, 'risk_id' : risk_id, 'idt' : idt, 'name' : name, 'rate' : rate}
attr = ['Cust_Id', 'name', 'risk_id', 'rate','idt']
for i in range(cust.shape[0]):#所有行 
    #uid pre_type Customer
    uid = prefix + str(cust.iloc[i,5]) + '>' #cust.iloc[i,5] is a uid，每个客户一个uid
    sub.append(uid)
    pre.append(pre_type)
    obj.append(prefix + 'Customer>')
    #生成属性
    value_df = cust.iloc[i,[0,1,3,4]]#没有2 idt属性，是因为，idt被单独拿出来做了一个Industry类
    value=value_df.astype(str)
    #value = value_df.tolist()
    #value = map(str,value)
    tupleInsertBatch(uid, '"'+ value +'"', attr_dict, attr)
            
    #生成关系hasIdtTyp
    idt= cust.iloc[i,2][0:6]#从第三列字符串中切割前6个字符
    RowN= temp[(temp['idt_cd']==idt)].index.tolist()
    R=RowN[0]# 行数，uid对应行
    tupleInsert(uid, hasIdtTyp, prefix + temp.iloc[R,2] + '>')
print 'customer creating done'            
        
        

#############################（3）
#创建对象Finatial_report,
typ = prefix + 'Rpt_Typ_Cd>'
aud = prefix + 'Rpt_Aud_Ind>'
clb = prefix + 'Rpt_Clb_Cd>'
rpt_id=prefix + 'Rpt_Id>'
dt = prefix + 'Rpt_Dt>'
prd = prefix + 'Rpt_Prd_Typ_Cd>'
updt = prefix + 'Rpt_Upd_Dt>'
        
rpt_bal = Fina[['Rpt_Id','Rpt_Clb_Cd', 'Rpt_Prd_Typ_Cd', 'Rpt_Dt', 'Aud_Ind', 'Upd_Dt', 'Rpt_Afl_Mdl','uidRpt']]
attr_dict = {'type':typ, 'aud':aud, 'clb':clb, 'dt':dt, 'prd':prd, 'updt':updt, 'rpt_id':rpt_id}
attr = ['rpt_id','clb', 'prd', 'dt', 'aud', 'updt', 'type']
#print rpt_bal
        
for i in range(rpt_bal.shape[0]):
    uid = prefix + str(rpt_bal.iloc[i,7]) + '>'#uid在td81_data预处理完的表中    
    sub.append(uid)
    pre.append(pre_type)
    obj.append(prefix + 'Balance_Sheet>')

    value_df = rpt_bal.iloc[i,0:7] 
    value=value_df.astype(str)
    #value = value_df.tolist()
    #tupleInsertBatch(uid, value, attr_dict, attr)
    tupleInsertBatch(uid, '"'+ value +'"', attr_dict, attr)#value[i]对应attr[i]
    
df1 = cust[['Cust_Id', 'uidCust']]
df2 = Fina[['Cust_Id', 'uidRpt']]
rel = pd.merge(df1, df2)#默认合并时候取并集
#print rel.shape
#print rel
for i in range(rel.shape[0]):
    subject = prefix + str(rel.iloc[i,1]) + '>'#1代表合并后的第2列
    predicate = prefix + 'hasFinRpt>'
    objectx = prefix + str(rel.iloc[i,2]) + '>'
    tupleInsert(subject, predicate, objectx)#添加tuple into graph
print 'financial report creating done' 
##############################（4）
#创建对象Financial_Subject
end_val =prefix +'Fin_Sub_End_Tm_Val>'
sub_name =prefix +'Fin_Sub_Nm>'
hasSub = prefix + 'hasFinSub>'

sub_bal = Fina[['tot_lia', 'tot_ass', 'la_rate', 'uidRpt', 'uidLia', 'uidAss', 'uidLarate']]#Fina由td81_data预处理
        
#print sub_bal.dtypes
        
for i in range(sub_bal.shape[0]):
    #print sub_bal.loc[i, 'tot_lia'].astype(float)
    fin_sub = prefix + 'Financial_Subject>'
    uidlst = []
    for j in range(3,7):
        uid = prefix + str(sub_bal.iloc[i,j]) + '>'#sub_bal.iloc[i,j]  is uid
        uidlst.append(uid)
    for k in range(1,4):#
        tupleInsert(uidlst[0], hasSub, uidlst[k])
        tupleInsert(uidlst[k], pre_type, fin_sub)
        if k==1:
            tupleInsert(uidlst[k], sub_name, '"'+'负债合计'+'"')
            sub_tot_lia=sub_bal.loc[i, 'tot_lia'].astype(str)
            tupleInsert(uidlst[k], end_val,'"'+ sub_tot_lia +'"')
        if k==2:
            tupleInsert(uidlst[k], sub_name,'"'+'负债所有者权益'+'"')
            sub_tot_ass=sub_bal.loc[i, 'tot_ass'].astype(str)
            tupleInsert(uidlst[k], end_val, '"'+ sub_tot_ass+'"')
        if k==3:
            tupleInsert(uidlst[k], sub_name, '"'+'资产负债率'+'"')
            sub_la_rate=sub_bal.loc[i, 'la_rate'].astype(str)
            tupleInsert(uidlst[k], end_val, '"'+sub_la_rate +'"')
print 'financial subject creating done' 
        
#################（4）
#输出文件
tuple_pre= pd.DataFrame([sub, pre, obj])
final= tuple_pre.T
final['end'] = '.'
final.to_csv('triple_final.nt', index = False, sep = ' ',quotechar="'", header = False, encoding='utf-8')
#to_csv中'quotechar'参数是：单字符无符号包含时，会自动添加起quotechar所指定的符号包含
print 'output file... \nEnd!'


      DW_Dat_Dt     Cust_Id  Rpt_Id Row_Id Itm_Id             Cust_Nm Rpt_Nm  \
0    2017/12/21   209000674       0   Null   Null          广州广日股份有限公司  资产负债表   
1    2017/12/21   219000111       0   Null   Null        中国宝武钢铁集团有限公司  资产负债表   
2    2017/12/21   219003710       0   Null   Null          唐山港陆钢铁有限公司  资产负债表   
3    2017/12/21   229000804       0   Null   Null      天津荣程联合钢铁集团有限公司  资产负债表   
4    2017/12/21   229000804       0   Null   Null      天津荣程联合钢铁集团有限公司  资产负债表   
5    2017/12/21   239000252       0   Null   Null          重庆钢铁股份有限公司  资产负债表   
6    2017/12/21   239001468       0   Null   Null         四川省川威集团有限公司  资产负债表   
7    2017/12/21   239001800       0   Null   Null          重庆秦安铸造有限公司  资产负债表   
8    2017/12/21   249000437       0   Null   Null      本溪钢铁（集团）有限责任公司  资产负债表   
9    2017/12/21   279000748       0   Null   Null          武汉钢铁股份有限公司  资产负债表   
10   2017/12/21   279001282       0   Null   Null          武汉钢铁（集团）公司  资产负债表   
11   2017/12/21  1109026373       0   Nu

industry creating done
customer creating done
financial report creating done
financial subject creating done
output file... 
End!


In [ ]:
# coding: utf-8
# encoding：utf-8
import numpy as np
import pandas as pd
import uuid
import time 
import logging

LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(filename='TD2Triple_python.log', level=logging.INFO, format=LOG_FORMAT)
 
    
def getUid():
    code = str(uuid.uuid1())
    abc = code.split('-')
    delimiter = ''
    code = delimiter.join(abc) 
    return code

sub = []
pre = []
obj = []

def tupleInsertBatch(uid, value, attr_dict, attr):
    for i in range(len(value)):
        sub.append(uid)
        pre.append(attr_dict[attr[i]])
        obj.append(value[i])
        
def tupleInsert(subject, predicate, objectx):
    sub.append(subject)
    pre.append(predicate)
    obj.append(objectx)
    
td81_data= pd.read_csv('td81.csv', header = 0)       
cust=pd.read_csv('customer.csv', header = 0)
cust.columns = ["Cust_Id","name","idt","risk_id","rate"]
        
prefix= '<http://www.cmbchina.com/bigdata/financial_risk#'
pre_type = '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>'
        

############校验
#创建异常类并初始化 
class DatapreproError(Exception):
    def __init__ (self,err):
                Exception.__init__(self,err)
                self.err=err 
    
'''
def Combine_check(ary):
    for i in range (ary.shape[0]):
                if ary[i]!=0:
                    raise DatapreproError('The tot_lia and tot_ass from TD are mismatch at %d row' %i)
                else:
                    pass
    print 'table is ready'

def Max_prio_check(table):
    rel=table.duplicated('Cust_Id',keep='first')
    for i in range (table.shape[0]):
        if rel[i]==1:
            raise DatapreproError("idxmax func error")
        else:
            pass 
    print 'max priority selecting check done'
'''
        
####################（0）
#########pre-process
#数据处理
#表一
td_1=td81_data[td81_data.Row_Id_Nm.isin(['负债合计'])]
td_1=td_1.reset_index(drop=True)
td_1_result= td_1.sort_values(by=['Cust_Id','Rpt_Id'])
td_1['tot_lia']=td_1_result['End_Tm_Val']#负债合计
#表二
td_2=td81_data[td81_data.Row_Id_Nm.isin(['负债和所有者权益']or['负债和所有者权益()'])]
td_2=td_2.reset_index(drop=True)
td_2_result= td_2.sort_values(by=['Cust_Id','Rpt_Id'])
td_2['tot_ass']=td_2_result['End_Tm_Val']#负债和所有者权益 
#新表
td= pd.concat([td_1, td_2['tot_ass']],axis=1)               

ary_rpt=np.array(td_1['Rpt_Id']-td_2['Rpt_Id']).T
print ary_rpt
try :
    ary=np.array(td_1['Cust_Id']-td_2['Cust_Id']).T
    for i in range (ary.shape[0]):
                if ary[i]!=0:
                    raise DatapreproError('error')           
except DatapreproError:
    logging.info('The data (tot_lia and tot_ass) splitting  from TD table failed')
    raise DatapreproError('The tot_lia and tot_ass from TD are mismatch at %d row' %i)
else:
    print 'Table is ready for calculation'

td['priority']='a'
td.loc[td['Rpt_Clb_Cd']=="合并",'priority']='2'
td.loc[td['Rpt_Clb_Cd']=="汇总",'priority']='1'
td.loc[td['Rpt_Clb_Cd']=="本部",'priority']='0'
td.loc[td['Aud_Ind']=="是",'priority']=td['priority'] + '1'
td.loc[td['Aud_Ind']=="否",'priority']=td['priority'] + '0'
td.loc[td['Rpt_Afl_Mdl']=="new",'priority']=td['priority'] + '1'
td.loc[td['Rpt_Afl_Mdl']=="old",'priority']=td['priority'] + '0'
td['priority']=td['priority'].astype(int)

td = td.loc[td.groupby('Cust_Id')['priority'].idxmax()].reset_index()
del td['priority']

try :
    rel=td.duplicated('Cust_Id',keep='first')
    for i in range (td.shape[0]):
        if rel[i]==False:#True
            raise DatapreproError("idxmax func error")
except DatapreproError:
    logging.info('Max priority selecting for calculation of liabilities ratio failed,idxmax func error')
    raise DatapreproError("idxmax func error")

else :
    print 'Max priority selecting check done'


tsf_lia=td['tot_lia'].str.replace(",","").astype(float)
tsf_ass=td['tot_ass'].str.replace(",", "").astype(float)

td['la_rate']=tsf_lia/tsf_ass#25
print 'liabilities ratio calculating done'

td['la_rate']=td['la_rate'].astype(float)
td['tot_lia']=tsf_lia 
td['tot_ass']=tsf_ass
td['uidRpt']='a'#26
td['uidLia']='a'
td['uidAss']='a'
td['uidLarate']='a'#29

for i in range(td.shape[0]):
    for j in range(26, 30):#26-29
              code = getUid()
              td.iloc[i,j] = code   
Fina=td


#######################（1）
#Industry 对象构建
idtcd=prefix + 'Idt_Cd>'
idtnm=prefix + 'Idt_Nm>'
idt_clea=cust['idt']
        
idt_clea=idt_clea.drop_duplicates()
idt_clea=idt_clea.reset_index(drop=True)
N=idt_clea.shape[0]
        
temp=pd.DataFrame(np.random.rand(N,1))
temp['idt_cd'],temp['idt_nm']=idt_clea.str.split(':',1).str
del temp[0]
temp["uidIdt"] ='a'    
for i in range(N):
    uid = prefix + str(code) + '>'
    temp.iloc[i,2] = code
    tupleInsert(uid, pre_type, prefix + 'IronSteel>')
    tupleInsert(uid,idtcd ,  '"'+temp.iloc[i,0]+'"') 
    tupleInsert(uid, idtnm, '"'+temp.iloc[i,1]+'"')
print 'industry creating done'

#########################（2）
#创建customer 对象
cust["uidCust"] = 'a' 
for i in range(cust.shape[0]):
    code = getUid()
    cust.iloc[i,5] = code
#属性property
Cust_Id = prefix + 'Cust_Core_Id>'
risk_id = prefix + 'Cust_Rsk_Id>'
idt = prefix + 'Cust_Idt_Typ>'
name = prefix + 'Cust_Nm>'
rate = prefix + 'Cust_Rat>'
#关系relation
hasIdtTyp = prefix + 'hasIdtTyp>'
attr_dict = {'Cust_Id' : Cust_Id, 'risk_id' : risk_id, 'idt' : idt, 'name' : name, 'rate' : rate}
attr = ['Cust_Id', 'name', 'risk_id', 'rate','idt']
for i in range(cust.shape[0]):
    uid = prefix + str(cust.iloc[i,5]) + '>' 
    sub.append(uid)
    pre.append(pre_type)
    obj.append(prefix + 'Customer>')
    #生成属性
    value_df = cust.iloc[i,[0,1,3,4]]
    value=value_df.astype(str)
    tupleInsertBatch(uid, '"'+ value +'"', attr_dict, attr)         
    #生成关系hasIdtTyp
    idt= cust.iloc[i,2][0:6]
    RowN= temp[(temp['idt_cd']==idt)].index.tolist()
    R=RowN[0]
    tupleInsert(uid, hasIdtTyp, prefix + temp.iloc[R,2] + '>')
print 'customer creating done'            
        
#############################（3）
#创建对象Finatial_report,
typ = prefix + 'Rpt_Typ_Cd>'
aud = prefix + 'Rpt_Aud_Ind>'
clb = prefix + 'Rpt_Clb_Cd>'
rpt_id=prefix + 'Rpt_Id>'
dt = prefix + 'Rpt_Dt>'
prd = prefix + 'Rpt_Prd_Typ_Cd>'
updt = prefix + 'Rpt_Upd_Dt>'
        
rpt_bal = Fina[['Rpt_Id','Rpt_Clb_Cd', 'Rpt_Prd_Typ_Cd', 'Rpt_Dt', 'Aud_Ind', 'Upd_Dt', 'Rpt_Afl_Mdl','uidRpt']]
attr_dict = {'type':typ, 'aud':aud, 'clb':clb, 'dt':dt, 'prd':prd, 'updt':updt, 'rpt_id':rpt_id}
attr = ['rpt_id','clb', 'prd', 'dt', 'aud', 'updt', 'type']       
for i in range(rpt_bal.shape[0]):
    uid = prefix + str(rpt_bal.iloc[i,7]) + '>'
    sub.append(uid)
    pre.append(pre_type)
    obj.append(prefix + 'Balance_Sheet>')
    value_df = rpt_bal.iloc[i,0:7] 
    value=value_df.astype(str)
    tupleInsertBatch(uid, '"'+ value +'"', attr_dict, attr)
    
df1 = cust[['Cust_Id', 'uidCust']]
df2 = Fina[['Cust_Id', 'uidRpt']]
rel = pd.merge(df1, df2)
for i in range(rel.shape[0]):
    subject = prefix + str(rel.iloc[i,1]) + '>'
    predicate = prefix + 'hasFinRpt>'
    objectx = prefix + str(rel.iloc[i,2]) + '>'
    tupleInsert(subject, predicate, objectx)
print 'financial report creating done' 

##############################（4）
#创建对象Financial_Subject
end_val =prefix +'Fin_Sub_End_Tm_Val>'
sub_name =prefix +'Fin_Sub_Nm>'
hasSub = prefix + 'hasFinSub>'
sub_bal = Fina[['tot_lia', 'tot_ass', 'la_rate', 'uidRpt', 'uidLia', 'uidAss', 'uidLarate']]       
for i in range(sub_bal.shape[0]):
    fin_sub = prefix + 'Financial_Subject>'
    uidlst = []
    for j in range(3,7):
        uid = prefix + str(sub_bal.iloc[i,j]) + '>'
        uidlst.append(uid)
    for k in range(1,4):#
        tupleInsert(uidlst[0], hasSub, uidlst[k])
        tupleInsert(uidlst[k], pre_type, fin_sub)
        if k==1:
            tupleInsert(uidlst[k], sub_name, '"'+'负债合计'+'"')
            sub_tot_lia=sub_bal.loc[i, 'tot_lia'].astype(str)
            tupleInsert(uidlst[k], end_val,'"'+ sub_tot_lia +'"')
        if k==2:
            tupleInsert(uidlst[k], sub_name,'"'+'负债所有者权益'+'"')
            sub_tot_ass=sub_bal.loc[i, 'tot_ass'].astype(str)
            tupleInsert(uidlst[k], end_val, '"'+ sub_tot_ass+'"')
        if k==3:
            tupleInsert(uidlst[k], sub_name, '"'+'资产负债率'+'"')
            sub_la_rate=sub_bal.loc[i, 'la_rate'].astype(str)
            tupleInsert(uidlst[k], end_val, '"'+sub_la_rate +'"')
print 'financial subject creating done' 
        
#################（4）
#输出文件
tuple_pre= pd.DataFrame([sub, pre, obj])
final= tuple_pre.T
final['end'] = '.'
final.to_csv('TD2Triple.nt', index = False, sep = ' ',quotechar="'", header = False, encoding='utf-8')
print 'output file... \nEnd!'




In [ ]:
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(filename)s[:%(lineno)d]- %(message)s"
logging.basicConfig(filename='example.log', level=logging.INFO, format=LOG_FORMAT)
try:
    a=3
    raise
except:
    logging.error('',exc_info=True)
    
print'hahhah '

In [16]:
import pandas as pd
import numpy as np
import random
a= pd.read_csv('td81.csv', header = 0)  
#print a.head(5)
cust=pd.read_csv('customer.csv', header = 0)
clb=['合并','汇总','本部']
mdl=['old','new']

Max1=404526278414.83
Min1=207278.11
Max2=741683433816.24
Min2=22680853.28
for i in range (a.shape[0]):
    a.loc[i,'Rpt_Clb_Cd']=random.choice(clb)
    #a.loc[i,'Rpt_Prd_Typ_Cd']=random.list
    a.loc[i,'Rpt_Afl_Mdl']=random.choice(mdl)
    a.loc[i,'Cust_Id']=a.loc[i,'Cust_Id'].astype(int)+1
    if a.loc[i,'Row_Id_Nm']=='负债合计':
        a.loc[i,'End_Tm_Val']=random.uniform(Min1,Max1)
    else:
        a.loc[i,'End_Tm_Val']=random.uniform(Min2,Max2)
#print a.head(5)
a.to_csv('T81_04.csv', index = False, encoding='utf-8')


In [11]:
import pandas as pd
import numpy as np
import random
import time 
TM=time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())
print TM
#df1 = pd.DataFrame(random.randint(3,4),index=range(0,4,1),columns=list('abcd'))
df2 = pd.DataFrame(np.arange(12).reshape(3,4),
                      columns=['A', 'B', 'C', 'D'])
#print df2
a=df2.index.tolist()

df1=pd.DataFrame(index=df2.index,columns=df2.columns)
print df1


2018-02-06 12:12:40
     A    B    C    D
0  NaN  NaN  NaN  NaN
1  NaN  NaN  NaN  NaN
2  NaN  NaN  NaN  NaN


In [13]:
import pandas as pd
import numpy as np

a= pd.read_csv('td81.csv', header = 0) 
a.loci,'']=
a.to_csv('T8_.csv', index = False, sep = ' ',quotechar="'", header = False, encoding='utf-8')

SyntaxError: can't assign to function call (<ipython-input-13-02049c968671>, line 5)

In [15]:
with open ('error_record.txt','a') as f:
                    f.write(':\t'+"cust_id"+"\t"+"rep_id"+"The No."+"mismatch error"+"\n")